In [1]:
from data_generators import get_train_test_addition

#dimensjoner og størrelser til x og y
n_digits = 2
n_max = 3*n_digits
m = 10

#definerer størrelsen på parametermatrisene
d = 15
k = 5
p = 20

#henter treningsdata
data = get_train_test_addition(n_digits,samples_per_batch=100,n_batches_train=4)

In [2]:
from utils import onehot

x = data['x_train'][0]
X = onehot(x,m)
y = data['y_train'][0]

In [3]:
data['x_train'].shape

(4, 100, 6)

In [4]:
from layers import LinearLayer,EmbedPosition,Attention,Softmax,CrossEntropy,FeedForward

embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

In [5]:
#vi ser at dLdW_2 tilhørende ff1 (først finner vi det lineære laget l2
#så nøkkel 'w' i params dict-en, så nøkkel 'd' for å finne den deriverte)
#nå er denne bare en nullmatrise 
ff1.l2.params['w']['d']

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [6]:
#"manuelt" forward pass (tilsvarende algoritme 1)
z0 = embed.forward(X)
z1 = att1.forward(z0)
z2 = ff1.forward(z1)
z = un_embed.forward(z2)
Z = softmax.forward(z)

In [7]:
#evaluerer objektfunksjonen
L = loss.forward(Z,y)
print(L)

#finner den deriverte av objektfunksjonen mhp Z
dLdZ = loss.backward()

1.9007710898666448


In [8]:
#"manuelt" backward pass (tilsvarende algoritme 2)
dLdz = softmax.backward(dLdZ)
print('1')
dLdz2 = un_embed.backward(dLdz)
print('2')
dLdz1 = ff1.backward(dLdz2)
print('3')
dLdz0 = att1.backward(dLdz1)
print('4')
embed.backward(dLdz0)

1
2
3
4


In [9]:
#etter backward pass har dLdW_2 fått verdier
ff1.l2.params['w']['d']

array([[-9.31302036e-03, -7.78563884e-03, -5.31467154e-03,
        -3.06476149e-03, -9.87308398e-03, -5.34037714e-03,
        -1.39988929e-02, -6.71750209e-03, -4.06823232e-03,
        -1.87126241e-02, -6.82591148e-03, -6.13680596e-03,
        -8.44116421e-03, -1.19555055e-02, -6.11249794e-03,
        -5.26043779e-03, -2.42859231e-03, -7.06331154e-03,
        -7.32015086e-03, -1.17951395e-02],
       [ 2.08125401e-03,  1.93547960e-03,  1.07965174e-03,
         6.83997348e-04,  2.23864213e-03,  1.06941532e-03,
         3.01365723e-03,  1.59921652e-03,  9.43065110e-04,
         4.14817778e-03,  1.53097418e-03,  1.28241110e-03,
         1.86259158e-03,  2.88737567e-03,  1.36843953e-03,
         1.36260173e-03,  4.52138385e-04,  1.40493357e-03,
         1.61299995e-03,  2.54720331e-03],
       [-5.64644618e-03, -4.93780064e-03, -3.23882630e-03,
        -1.64962582e-03, -6.00421948e-03, -2.67912532e-03,
        -7.83796952e-03, -4.04616234e-03, -2.42155707e-03,
        -1.03236067e-02, -4.1

In [10]:
from neural_network import NeuralNetwork

#vi kan samle lagene i en liste som vi bruker for å
#initialisere et nevralt nettverk der vi kan bruke forward() og backward() 
#for å oppnå det samme som vi gjorde manuelt over

layers = [embed,att1,ff1,un_embed,softmax]
nn = NeuralNetwork(layers)

In [11]:
#forward pass tilsvarende algoritme 1
Z = nn.forward(X)

#beregner loss med CrossEntropy
L = loss.forward(Z,y)
print(L)

#backward pass tilsvarende algoritme 2
dLdZ = loss.backward()
nn.backward(dLdZ)

1.9007710898666448


In [12]:
#før optimering er W_2[0,0] gitt ved
#ff1 er det tredje laget i layers-listen (derav layers[2])
W_2_pre_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_pre_opt[0,0])

-0.10365973268938658


In [13]:
import numpy as np

xs = data['x_train']
ys = data['y_train']

n_batches = xs.shape[0]
n_iters = 100
step_size = 0.1

#treningsløkke tilsvarende algoritme 4 (med gradient descent)
for j in range(n_iters):
    losses = []
    for i in range(n_batches):
        x = xs[i]
        y = ys[i]

        X = onehot(x,m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_gd(step_size)
    mean_loss = np.mean(losses)
    print("Iterasjon ", str(j), " L = ",mean_loss, "")

c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:136: RuntimeWarning: overflow encountered in exp
  self.P = np.exp(x)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:138: RuntimeWarning: invalid value encountered in divide
  self.Z = self.P/(Q+self.epsilon)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:147: RuntimeWarning: overflow encountered in exp
  P = np.exp(self.x)
c:\Users\tryme\Documents\GitHub\TMA4320-Introduksjon-til-vitenskapelige-beregninger-prosjekter\Prosjekt2-IndustriellMatematikk\layers.py:149: RuntimeWarning: invalid value encountered in divide
  S = P/(Q*Q+self.epsilon)


In [ ]:
#etter trening ser vi at W_2 er gitt ved

W_2_post_opt = nn.layers[2].l2.params['w']['w'].copy()
print(W_2_post_opt[0,0])

#dersom differansen er større enn null har dette parameteret endret seg etter 
#gradient descent
print(W_2_post_opt[0,0] - W_2_pre_opt[0,0])




-0.13857317601666091
-0.00010665015797989286
